In [1]:
# download data from Rfam
# run download_rfam.R
# gzip rfam

In [1]:
import os 
import random
import json
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupShuffleSplit

In [2]:
# basic random seed
DEFAULT_RANDOM_SEED = 2022

def seedBasic(seed=DEFAULT_RANDOM_SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
# tensorflow random seed 
#def seedTF(seed=DEFAULT_RANDOM_SEED):
#    tf.random.set_seed(seed)
    
# torch random seed
# import torch
# def seedTorch(seed=DEFAULT_RANDOM_SEED):
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.backends.cudnn.deterministic = True
#     torch.backends.cudnn.benchmark = False
      
# basic + tensorflow + torch 
def seedEverything(seed=DEFAULT_RANDOM_SEED):
    seedBasic(seed)
    #seedTF(seed)
    # seedTorch(seed)

seedEverything()

In [3]:
label='IRES'
ID='Seq.ID'
feature_seq='Seq'
feature_structure='secondary'
feature_group='ID'
feature_seq_len=600
feature_pad_end='3end'

df = pd.read_csv('input/rfam/rfam_seq_100_structure.csv.gz')
df['Seq.ID'] = df['Seq.ID'].str.replace('/','_')
df = df.drop_duplicates(subset=['Seq.ID'])

df

,Seq.ID,Accession,ID,Type,Description,Seq.Header,Seq,Seq.len,IRES,seq,secondary,MFE
0,AAAA02037773.1_7389-7271,RF00059,TPP,Cis-reg; riboswitch,TPP riboswitch (THI element),>AAAA02037773.1/7389-7271 Oryza sativa Indica ...,GCCGTCTCAACGGGGTGCGGAGAGTCGGGCCGCGTGATAAGCCCGA...,119,0,GCCGUCUCAACGGGGUGCGGAGAGUCGGGCCGCGUGAUAAGCCCGA...,((.(.(((((((((((((((((((((((((..........))))))...,-46.70
1,AAAA02038895.1_3910-4349,RF01766,cspA,Cis-reg; thermoregulator,cspA thermoregulator,>AAAA02038895.1/3910-4349 Oryza sativa Indica ...,ATCGGTTTGTAGTACAGACCTTATGAAAGCAGTTTTAGTAAAGCAG...,440,0,AUCGGUUUGUAGUACAGACCUUAUGAAAGCAGUUUUAGUAAAGCAG...,..(((((((..(((((((((((((.(((.(((((.(((..(((.((...,-103.87
2,AAAA02039262.1_995-1124,RF00391,RtT,Cis-reg,RtT RNA,>AAAA02039262.1/995-1124 Oryza sativa Indica G...,TTTTCGCCTGTACGTGAGAGGATGAGAATCTCCGAAGGAGGTTTGA...,130,0,UUUUCGCCUGUACGUGAGAGGAUGAGAAUCUCCGAAGGAGGUUUGA...,............(((((((((((..((((((((...)))))))).....,-44.10
3,AAAA02039442.1_263-1,RF01766,cspA,Cis-reg; thermoregulator,cspA thermoregulator,>AAAA02039442.1/263-1 Oryza sativa Indica Grou...,ATCGGTTTGGAATACAGACCTTATAAAAGCAGTTTAGTAAAGCAGT...,263,0,AUCGGUUUGGAAUACAGACCUUAUAAAAGCAGUUUAGUAAAGCAGU...,...((((((.....)))))).....(((.((.((((.((((.((((...,-61.50
4,AAAA02041097.1_4543-4435,RF01771,rnk_leader,Cis-reg; leader,Enterobacteria rnk leader,>AAAA02041097.1/4543-4435 Oryza sativa Indica ...,TTCATATGGGACATGAATGTGACCTTCATCACTTCAAACTTGAGCG...,109,0,UUCAUAUGGGACAUGAAUGUGACCUUCAUCACUUCAAACUUGAGCG...,...((((.((.(.(((((((.....(((............))))))...,-30.80
...,...,...,...,...,...,...,...,...,...,...,...,...
167576,Z82044.1_19193-19300,RF00059,TPP,Cis-reg; riboswitch,TPP riboswitch (THI element),>Z82044.1/19193-19300 Bacillus subtilis 25 kb ...,AATAGTTACTGGGGGTGCCCGCTTTCGGGCTGAGAGAGAAGGCAAG...,108,0,AAUAGUUACUGGGGGUGCCCGCUUUCGGGCUGAGAGAGAAGGCAAG...,....((((..((((.((((..(((((.......)))))..)))).....,-33.60
167577,Z84266.1_6-249,RF00061,IRES_HCV,Cis-reg; IRES,Hepatitis C virus internal ribosome entry site,>Z84266.1/6-249 Hepatitis C virus 5' noncoding...,GCAGAAAGCGTCTAGCCATGGCGTTAGTATGAGTGTCGTGCAGCCT...,244,1,GCAGAAAGCGUCUAGCCAUGGCGUUAGUAUGAGUGUCGUGCAGCCU...,......((((((.......)))))).((((((...)))))).((((...,-93.40
167578,Z84276.1_6-250,RF00061,IRES_HCV,Cis-reg; IRES,Hepatitis C virus internal ribosome entry site,>Z84276.1/6-250 Hepatitis C virus 5' noncoding...,GCAGAAAGCGTCTAGCCATGGCGTTAGTATGAGTGTCGTACAGCCT...,245,1,GCAGAAAGCGUCUAGCCAUGGCGUUAGUAUGAGUGUCGUACAGCCU...,......((((((.......)))))).((((((...)))))).((((...,-94.30
167579,Z84281.1_6-249,RF00061,IRES_HCV,Cis-reg; IRES,Hepatitis C virus internal ribosome entry site,>Z84281.1/6-249 Hepatitis C virus 5' noncoding...,GCAGAAAGCGTCTAGCCATGGCGTTAGTATGAGTGTCGTGCAGCCT...,244,1,GCAGAAAGCGUCUAGCCAUGGCGUUAGUAUGAGUGUCGUGCAGCCU...,......((((((.......)))))).((((((...)))))).((((...,-89.50


In [4]:
df.IRES.value_counts()

0    166138
1      1443
Name: IRES, dtype: int64

In [5]:
y=df[label]
#y=tf.keras.utils.to_categorical(y)
g=df[feature_group]
#print(X.shape)
print(y.shape)
add_on=['A','C','G','T']
add_on_str = ['.']

(167581,)


In [ ]:
# prepare data
data=df[[ID,feature_seq,feature_structure,feature_group,label]]
for fold in range(1,11):
    print(fold)
    
    valid_split = GroupShuffleSplit(test_size=.20, n_splits=2)
    
    while True:
        split = valid_split.split(data, y, groups=g)
        train_inds, valid_inds = next(split)
        if len(train_inds)/len(valid_inds)>3:
            break
    
    data_train=data.iloc[train_inds,:]
    data_valid=data.iloc[valid_inds,:]
    
    # data augmentation for postive examples in training set; add or remove 1-3nt
    data_train_pos=data_train[data_train.IRES==1]
    data_train_pos_new = data_train_pos.sample(n=data_train.shape[0], random_state=1, replace=True)
    
    for i in range(data_train_pos_new.shape[0]):
        r=random.uniform(0, 1)
        if r>0.75:
            tmp=random.sample([-1, -2, -3], k=1)
            removed_str = data_train_pos_new[feature_structure].iloc[i][tmp[0]:]
            if '(' in removed_str or ')' in removed_str:
                continue
            else:
                data_train_pos_new[feature_seq].iloc[i]=data_train_pos_new[feature_seq].iloc[i][:tmp[0]]
                data_train_pos_new[feature_structure].iloc[i]=data_train_pos_new[feature_structure].iloc[i][:tmp[0]]
        if r >0.5 and r <=0.75:
            tmp=random.sample([1, 2, 3], k=1)
            removed_str = data_train_pos_new[feature_structure].iloc[i][:tmp[0]]
            if '(' in removed_str or ')' in removed_str:
                continue
            else:
                data_train_pos_new[feature_seq].iloc[i]  =data_train_pos_new[feature_seq].iloc[i][tmp[0]:]
                data_train_pos_new[feature_structure].iloc[i]=data_train_pos_new[feature_structure].iloc[i][tmp[0]:]
        if r >0.25 and r <=0.5:
            tmp=''.join(np.random.choice(add_on, size=random.sample([1,2,3],k=1), replace=True))
            tmp_str=re.sub(r"[A-Z]", '.', tmp)
            data_train_pos_new[feature_seq].iloc[i]=data_train_pos_new[feature_seq].iloc[i] + tmp
            data_train_pos_new[feature_structure].iloc[i]=data_train_pos_new[feature_structure].iloc[i] + tmp_str
        else:
            tmp=''.join(np.random.choice(add_on, size=random.sample([1,2,3],k=1), replace=True))
            tmp_str=re.sub(r"[A-Z]", '.', tmp)
            data_train_pos_new[feature_seq].iloc[i]=tmp + data_train_pos_new[feature_seq].iloc[i]
            data_train_pos_new[feature_structure].iloc[i]=tmp_str + data_train_pos_new[feature_structure].iloc[i]

    data_train_all = pd.concat([data_train, data_train_pos_new])
    data_train_all['set']='train'
    data_train_all=data_train_all.drop_duplicates()
    data_valid['set']='valid'
    
    data_all = pd.concat([data_train_all, data_valid])
    data_all = data_all.sample(frac=1).reset_index(drop=True)

    data_all.to_csv('input/fold'+str(fold)+'.csv', index=False)


1


/tmp/ipykernel_3702550/4002073596.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train_pos_new[feature_seq].iloc[i]=tmp + data_train_pos_new[feature_seq].iloc[i]
/tmp/ipykernel_3702550/4002073596.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train_pos_new[feature_structure].iloc[i]=tmp_str + data_train_pos_new[feature_structure].iloc[i]
/tmp/ipykernel_3702550/4002073596.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

2


/tmp/ipykernel_3702550/4002073596.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train_pos_new[feature_seq].iloc[i]=data_train_pos_new[feature_seq].iloc[i] + tmp
/tmp/ipykernel_3702550/4002073596.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train_pos_new[feature_structure].iloc[i]=data_train_pos_new[feature_structure].iloc[i] + tmp_str
/tmp/ipykernel_3702550/4002073596.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

3


/tmp/ipykernel_3702550/4002073596.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train_pos_new[feature_seq].iloc[i]=data_train_pos_new[feature_seq].iloc[i] + tmp
/tmp/ipykernel_3702550/4002073596.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train_pos_new[feature_structure].iloc[i]=data_train_pos_new[feature_structure].iloc[i] + tmp_str
/tmp/ipykernel_3702550/4002073596.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

4


/tmp/ipykernel_3702550/4002073596.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train_pos_new[feature_seq].iloc[i]=tmp + data_train_pos_new[feature_seq].iloc[i]
/tmp/ipykernel_3702550/4002073596.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train_pos_new[feature_structure].iloc[i]=tmp_str + data_train_pos_new[feature_structure].iloc[i]
/tmp/ipykernel_3702550/4002073596.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

In [ ]:
! gzip input/*.csv